In [2]:
import argparse
import os
import sys
import time
import numpy as np
from PIL import Image

import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import urllib
import time

This is following the model from LucidRains with some advisement from the original code

https://github.com/lucidrains/ESBN-pytorch

# helper functions

In [3]:
def exists(val):
    return val is not None

In [4]:
def safe_cat(t, el, dim =0):
    if not exists(t):
        return el
    return tf.concat((t, el), axis = dim)

In [5]:
def map_fn(fn, *args, **kwargs):
    def inner(*arr):
        return map(lambda t: fn(t, *args, **kwargs), arr)
    return inner

# the Class

In [7]:
class ESBN(keras.layers.Layer):
    def __init__(
        self,
        *,
        value_dim = 64,
        key_dim = 64,
        hidden_dim = 512,
        output_dim = 4,
        encoder = None
    ):
        super().__init__()
        self.h0 = tf.zeros(hidden_dim)
        self.c0 = tf.zeros(hidden_dim)
        self.k0 = tf.zeros(key_dim + 1)
        
        self.rnn = tf.keras.layers.LSTMCell(hidden_dim)  #What is the difference between this and just LSTM
        self.to_gate = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid) #?
        self.to_key = tf.keras.layers.Dense(key_dim)
        self.to_output = tf.keras.layers.Dense(output_dim)
        
        self.encoder = tf.keras.Sequential()
        self.encoder.add(tf.keras.layers.Conv2D(32, kernel_size=4, strides=2, 
                                                activation=tf.keras.activations.relu))
        self.encoder.add(tf.keras.layers.Conv2D(64, kernel_size=4, strides=2,
                                                activation=tf.keras.activations.relu))
        self.encoder.add(tf.keras.layers.Conv2D(64, kernel_size=4, strides=2,
                                                activation=tf.keras.activations.relu))
        self.encoder.add(tf.keras.layers.Flatten())
        self.encoder.add(tf.keras.layers.Dense(value_dim))
                         # if not exists(encoder) else encoder What does???
            
        self.to_confidence = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)
        
    def forward(self, images):
        b = images.shape[1]
        Mk = None
        Mv = None
        
        hx, cx, kx, k0 = map_fn(repeat, 'd -> b d', b = b)(self.h0, self.c0, self.k0, self.k0)
        out =[] #note, there could be issues with lists. aslo consider a better way becaues other languages bla bla
        
        for ind, image in enumerate(images):
            is_first = ind == 0
            z = self.encoder(image)
            hx, cx = self.rnn(kx, (hx, cx)) #return state?
            y, g, kw = self.to_output(hx), self.to_gate(hx), self.to_key(hx)
            
            # if is_first: #redundent?
            #     kx = k0
            # else:
            if not is_first:
                # attention
                sim = tf.keras.layers.EinsumDense('b n d, b d -> b n',
                                                  activations=tf.keras.activations.linear)([Mv, z])
                wk = tf.keras.layers.Activation(activation=tf.keras.activations.softmax)(sim)
                # sim = tf.einsum('b n d, b d -> b n', Mv, z)
                # wk = sim.tf.softmax(-1) #????
                
                # calculate confidence
                sim, wk = map_fn(rearrange, 'b n -> b n ()')(sim, wk) #consider
                ck = self.to_confidence(sim)
                
                
            
        
        
        
        
    